### European starling custom parsing
- An labelled (but smaller) dataset of zebra finch vocalizations
    - .WAV files with individual labels
- This notebook creates a JSON corresponding to each WAV file (and Noise file where available).
- Dataset origin:
    - https://zenodo.org/record/3237218

In [1]:
from avgn.utils.general import prepare_env

In [2]:
prepare_env()

env: CUDA_VISIBLE_DEVICES=GPU


### Import relevant packages

In [3]:
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm
import pandas as pd
pd.options.display.max_columns = None
import librosa
from datetime import datetime
import numpy as np

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
import avgn
from avgn.custom_parsing.starling_gentner import generate_json
from avgn.utils.paths import DATA_DIR

### Load data in original format

In [5]:
DATASET_ID = 'european_starling_gentner'

In [6]:
# create a unique datetime identifier for the files output by this notebook
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2019-06-27_13-49-37'

In [7]:
DSLOC = avgn.utils.paths.Path('/mnt/cube/Datasets/starling_zenodo/')
DSLOC

PosixPath('/mnt/cube/Datasets/starling_zenodo')

In [8]:
WAVLIST = list((DSLOC).expanduser().glob('*/wavs/*.wav'))
len(WAVLIST), WAVLIST[0]

(6932,
 PosixPath('/mnt/cube/Datasets/starling_zenodo/b1066/wavs/2017-11-09_13-40-55-270000.wav'))

In [9]:
wav_df = pd.DataFrame([[i.parent.parent.stem, i, i.stem] for i in WAVLIST], columns = ['indv', 'wavloc', 'wavdate'])
wav_df[:3]

,indv,wavloc,wavdate
0,b1066,/mnt/cube/Datasets/starling_zenodo/b1066/wavs/...,2017-11-09_13-40-55-270000
1,b1066,/mnt/cube/Datasets/starling_zenodo/b1066/wavs/...,2017-11-15_17-04-08-550000
2,b1066,/mnt/cube/Datasets/starling_zenodo/b1066/wavs/...,2017-11-07_15-38-52-610000


In [17]:
wav_df = wav_df[np.array([i.stem[0] != '.' for i in wav_df.wavloc.values])]

### generate JSON

In [19]:
for idx, row in tqdm(wav_df.iterrows(), total = len(wav_df)):
    generate_json(row, DT_ID)